将NHANES的饮食数据与FPED的数据关联，得出相应的食物的饮食模式，对应DGA中的饮食模式

In [1]:
import pandas as pd

In [2]:
# 导入数据
individual_firstday_df = pd.read_sas('e:/work/Paper/NHANES/0910Analysis Data/Diet/NHANES-Diet-data/DR1IFF_F.XPT')
individual_secondday_df = pd.read_sas('e:/work/Paper/NHANES/0910Analysis Data/Diet/NHANES-Diet-data/DR2IFF_F.XPT')

FPED_df = pd.read_excel('e:/work/Paper/NHANES/0910Analysis Data/Diet/FPED_0910.xls')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [3]:
# 查看有多少编码可以匹配上，多少不能匹配上，以及匹配上的比例
# 全部匹配上
individual_day1_usda = individual_firstday_df['DR1IFDCD'].tolist()
individual_day2_usda = individual_secondday_df['DR2IFDCD'].tolist()

FOODCODE_in_FPED = FPED_df['FOODCODE'].to_list()

matched_counts = 0
unmatched_counts = 0 

for m in [individual_day1_usda, individual_day2_usda]:
    for q in individual_day1_usda:
        if q in FOODCODE_in_FPED:
            matched_counts += 1
        else:
            unmatched_counts += 1
    print('匹配成功:',matched_counts, '匹配不成功:',unmatched_counts, '比例:',matched_counts/(matched_counts+unmatched_counts))

匹配成功: 150991 匹配不成功: 0 比例: 1.0
匹配成功: 301982 匹配不成功: 0 比例: 1.0


In [4]:
# 合并两天数据
individual_concat_df = pd.concat([individual_firstday_df, individual_secondday_df])
individual_concat_df.reset_index(drop=True, inplace=True)
individual_concat_df.shape

(280132, 165)

In [5]:
#提取FPED中DGA中的相关变量
FPED_feature = [
    'FOODCODE',
    'F_TOTAL (cup eq.)', 'F_JUICE (cup eq.)',                                           #Fruits
    'V_TOTAL (cup eq.)',  'V_DRKGR (cup eq.)',                                          #Vegetables
    'PF_LEGUMES (oz. eq.)' ,                                                            #LEGUMES
    'G_WHOLE (oz. eq.)', 'G_REFINED (oz. eq.)',                                         #Grain
    'D_TOTAL (cup eq.)',                                                                #Dairy
    'PF_SEAFD_HI (oz. eq.)','PF_SEAFD_LOW (oz. eq.)', 'PF_TOTAL (oz. eq.)',             #Protein Foods
    'PF_NUTSDS (oz. eq.)', 'PF_SOY (oz. eq.)',                                      
    'ADD_SUGARS (tsp. eq.)'   
]

FPED_extract_df = FPED_df[FPED_feature]

In [6]:
# 提取饮食访谈中的食物编码及克重
individual_feature = [
    'SEQN',
    'DR1IFDCD', #USDA编码
    'DR2IFDCD', #USDA编码
    'DR1IGRMS', #克重
    'DR2IGRMS', #克重
]
# individual_extract_df 从合并两天数据中提取出的变量(individual_feature)
individual_extract_df = individual_concat_df[individual_feature]
individual_extract_df.fillna(0, inplace=True)   #填补缺失值，用于加和

individual_extract_df.loc[:,'DRIGRMS'] = individual_extract_df.loc[:, 'DR1IGRMS'] + individual_extract_df.loc[:, 'DR2IGRMS']    #合并克重为新变量DRIGRMS
individual_extract_df.loc[:,'DRIGRMS/100'] = individual_extract_df.loc[:,'DRIGRMS']/100 #由DRIGRMS除以100得出新的变量DRIGRMS/100 进行单位换算
individual_extract_df.loc[:,'DRIFDCD'] = individual_extract_df.loc[:, 'DR1IFDCD'] + individual_extract_df.loc[:, 'DR2IFDCD']    #和合并USDA编码为新变量DRIFDCD
individual_extract_df = individual_extract_df.loc[:, ['SEQN', 'DRIGRMS', 'DRIGRMS/100', 'DRIFDCD']]

C:\Users\Gstar\AppData\Local\Temp\ipykernel_20804\591831136.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_extract_df.fillna(0, inplace=True)   #填补缺失值，用于加和
C:\Users\Gstar\AppData\Local\Temp\ipykernel_20804\591831136.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_extract_df.loc[:,'DRIGRMS'] = individual_extract_df.loc[:, 'DR1IGRMS'] + individual_extract_df.loc[:, 'DR2IGRMS']    #合并克重为新变量DRIGRMS
C:\Users\Gstar\AppData\Local\Temp\ipykernel_20804\591831136.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [7]:
#将每个食物的USDA编码与FPED中的编码匹配
individual_FPED_merge_df = pd.merge(individual_extract_df, FPED_extract_df, left_on='DRIFDCD', right_on='FOODCODE', how='left')
individual_FPED_merge_df = individual_FPED_merge_df.drop(labels=['DRIFDCD'], axis=1)
individual_FPED_merge_df

,SEQN,DRIGRMS,DRIGRMS/100,FOODCODE,F_TOTAL (cup eq.),F_JUICE (cup eq.),V_TOTAL (cup eq.),V_DRKGR (cup eq.),PF_LEGUMES (oz. eq.),G_WHOLE (oz. eq.),G_REFINED (oz. eq.),D_TOTAL (cup eq.),PF_SEAFD_HI (oz. eq.),PF_SEAFD_LOW (oz. eq.),PF_TOTAL (oz. eq.),PF_NUTSDS (oz. eq.),PF_SOY (oz. eq.),ADD_SUGARS (tsp. eq.)
0,51624.0,1332.00,13.3200,92101000,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,51624.0,355.00,3.5500,92410350,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,51624.0,207.03,2.0703,63101000,0.91,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,51624.0,298.13,2.9813,61119010,0.54,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,51624.0,355.00,3.5500,92410350,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334304,62160.0,84.32,0.8432,41205010,0.00,0.0,0.06,0.0,2.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334305,62160.0,84.32,0.8432,41205010,0.00,0.0,0.06,0.0,2.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334306,62160.0,84.32,0.8432,41205010,0.00,0.0,0.06,0.0,2.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334307,62160.0,42.00,0.4200,63223020,0.69,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 构建食物对应DAG评分的组分 --  individual_DGA_standard_df  
individual_transformed_df = pd.DataFrame()          #individual_transformed_df  第一天和第二天合并数据经过FPED转换后的数据
individual_transformed_df['SEQN'] = individual_FPED_merge_df['SEQN']
individual_transformed_df['FOODCODE'] = individual_FPED_merge_df['FOODCODE']

FPED_feature = [
    'F_TOTAL (cup eq.)', 'F_JUICE (cup eq.)',                 #Fruits
    'V_TOTAL (cup eq.)',  'V_DRKGR (cup eq.)',                #Vegetables
    'PF_LEGUMES (oz. eq.)' ,                                  #LEGUMES
    'G_WHOLE (oz. eq.)', 'G_REFINED (oz. eq.)',               #Grain
    'D_TOTAL (cup eq.)',                                      #Dairy
    'PF_SEAFD_HI (oz. eq.)','PF_SEAFD_LOW (oz. eq.)',   'PF_TOTAL (oz. eq.)',         #Protein Foods
    'PF_NUTSDS (oz. eq.)', 'PF_SOY (oz. eq.)',
    'ADD_SUGARS (tsp. eq.)'
]

transformed_name = [
    'F_TOTAL', 'F_JUICE',                    #Fruits
    'V_TOTAL',  'V_DRKGR',                   #Vegetables
    'PF_LEGUMES' ,                           #LEGUMES 
    'G_WHOLE', 'G_REFINED',                  #Grain
    'D_TOTAL',                               #Dairy
    'PF_SEAFD_HI','PF_SEAFD_LOW', 'PF_TOTAL',           #Protein Foods
    'PF_NUTSDS', 'PF_SOY',
    'ADD_SUGARS'
    ]

for i,q in zip(transformed_name, FPED_feature):
    print(i)
    individual_transformed_df[i] = individual_FPED_merge_df['DRIGRMS/100']*individual_FPED_merge_df[q]



individual_DGA_standard_df = individual_transformed_df.groupby(by='SEQN').sum().drop('FOODCODE', axis=1)    
individual_DGA_standard_df = individual_DGA_standard_df/2             

F_TOTAL
F_JUICE
V_TOTAL
V_DRKGR
PF_LEGUMES
G_WHOLE
G_REFINED
D_TOTAL
PF_SEAFD_HI
PF_SEAFD_LOW
PF_TOTAL
PF_NUTSDS
PF_SOY
ADD_SUGARS


In [9]:
# 添加 第一天第二天总能量；饱和脂肪酸；单不饱和脂肪酸； 多不饱和脂肪酸； 糖分；
individual_firstday_total_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Diet/NHANES-Diet-data/DR1TOT_F.XPT')
individual_secondday_total_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Diet/NHANES-Diet-data/DR2TOT_F.XPT')

# 合并数据
individual_total_concat_df = pd.concat([individual_firstday_total_df, individual_secondday_total_df])
individual_total_concat_df.reset_index(drop=True, inplace=True)

In [10]:
# 构建参与者的  卡路里， 单不饱和脂肪酸， 多不饱和脂肪酸， 饱和脂肪酸， 钠 的摄入量    individual_total_df  每星期
# individual_total_df
individual_total_extract_features = [
    'SEQN',
    'DR1TKCAL', 'DR2TKCAL',     #卡路里
    'DR1TMFAT', 'DR2TMFAT',     #单不饱和脂肪酸
    'DR1TPFAT', 'DR2TPFAT',     #多不饱和脂肪酸
    'DR1TSFAT', 'DR2TSFAT',     #饱和脂肪酸'
    'DR1TSODI', 'DR2TSODI',     #钠
]

individual_total_merge_features = [
    'DRTKCAL',     #卡路里
    'DRTMFAT',     #单不饱和脂肪酸
    'DRTPFAT',     #多不饱和脂肪酸
    'DRTSFAT',     #饱和脂肪酸'
    'DRTSODI',     #钠
]
individual_total_extract_df = individual_total_concat_df[individual_total_extract_features]
individual_total_extract_df.fillna(0, inplace=True)

individual_total_df = pd.DataFrame()
individual_total_df['SEQN'] = individual_total_extract_df['SEQN']
individual_total_df['DRTKCAL'] = individual_total_extract_df['DR1TKCAL'] + individual_total_extract_df['DR2TKCAL']
individual_total_df['DRTMFAT'] = individual_total_extract_df['DR1TMFAT'] + individual_total_extract_df['DR2TMFAT']
individual_total_df['DRTPFAT'] = individual_total_extract_df['DR1TPFAT'] + individual_total_extract_df['DR2TPFAT']
individual_total_df['DRTSFAT'] = individual_total_extract_df['DR1TSFAT'] + individual_total_extract_df['DR2TSFAT']
individual_total_df['DRTSODI'] = individual_total_extract_df['DR1TSODI'] + individual_total_extract_df['DR2TSODI']

individual_total_df = individual_total_df.groupby(by='SEQN').sum()

individual_total_df = individual_total_df/2

C:\Users\Gstar\AppData\Local\Temp\ipykernel_20804\176251906.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_total_extract_df.fillna(0, inplace=True)


In [11]:
# 根据SEQN合并两份数据
HEI_evaluate_df = pd.merge(left=individual_DGA_standard_df, right=individual_total_df, on='SEQN', how='left')
HEI_evaluate_df.reset_index(drop=False, inplace=True)

In [12]:
# 根据HEI 进行评分
score_in_HEI = pd.DataFrame()
score_in_HEI['SEQN'] = HEI_evaluate_df['SEQN']

In [13]:
### Total Fruits评分
total_fruit_per_kcal = HEI_evaluate_df['F_TOTAL'] * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.8
min = 0
points = 5
for i in total_fruit_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Total Fruits'] = score_list

In [14]:
### Whole Fruits评分
whole_fruit_per_kcal = (HEI_evaluate_df['F_TOTAL'] - HEI_evaluate_df['F_JUICE']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.4
min = 0
points = 5
for i in whole_fruit_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)
score_in_HEI['Whole Fruit'] = score_list

In [15]:
### Total Vegetables评分
total_vegetables_per_kcal = (HEI_evaluate_df['V_TOTAL'] + HEI_evaluate_df['PF_LEGUMES']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 1.1
min = 0
points = 5
for i in total_vegetables_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Total Vegetables'] = score_list

In [16]:
### Greens and Beans评分
greens_and_beans_per_kcal = (HEI_evaluate_df['V_DRKGR'] + HEI_evaluate_df['PF_LEGUMES']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.2
min = 0
points = 5
for i in greens_and_beans_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Greens and Beans'] = score_list

In [17]:
### Whole Grains评分
whole_grains_per_kcal = (HEI_evaluate_df['G_WHOLE']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 1.5
min = 0
points = 10
for i in whole_grains_per_kcal:
    if i >= max:
        score = points
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Whole Grains'] = score_list

In [18]:
### Dairy评分
dairy_per_kcal = (HEI_evaluate_df['D_TOTAL']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 1.3
min = 0
points = 10
for i in dairy_per_kcal:
    if i >= max:
        score = points  
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Dairy'] = score_list

In [19]:
### Total Protein Foods评分
total_protein_foods_per_kcal = (HEI_evaluate_df['PF_TOTAL']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 2.5
min = 0
points = 5
for i in total_protein_foods_per_kcal:
    if i >= max:
        score = points    
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Total Protein'] = score_list

In [20]:
### Seafood and Plant Proteins评分
seafood_and_plant_proteins_per_kcal = (HEI_evaluate_df['PF_SEAFD_HI'] + HEI_evaluate_df['PF_SEAFD_LOW'] + HEI_evaluate_df['PF_LEGUMES']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.8
min = 0
points = 5
for i in seafood_and_plant_proteins_per_kcal:
    if i >= max:
        score = points    
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Seafood and Plant Proteins'] = score_list

In [21]:
### Fatty Acids评分
fatty_acids = (HEI_evaluate_df['DRTMFAT'] + HEI_evaluate_df['DRTPFAT']) /  HEI_evaluate_df['DRTSFAT']
score_list = []
max = 2.5
min = 1.2
points = 10
for i in fatty_acids:
    if i >= max:
        score = points    
        score_list.append(score)
    elif i <= min:
        score = 0
        score_list.append(score)
    else:
        score = points*(i-min)/(max - min)
        score_list.append(score)

score_in_HEI['Fatty Acids'] = score_list

In [22]:
### Refined Grains评分
refined_grains_per_kcal = (HEI_evaluate_df['G_REFINED']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 4.3
min = 1.8
points = 10
for i in refined_grains_per_kcal:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Refined Grains'] = score_list

In [23]:
### Sodium评分
sodium = (HEI_evaluate_df['DRTSODI']) * 1000 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 2.0
min = 1.1
points = 10
for i in sodium:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Sodium'] = score_list

In [24]:
### Added Sugars评分
added_sugars_per_kcal = (HEI_evaluate_df['ADD_SUGARS']) * 16 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.26
min = 0.065
points = 10
for i in added_sugars_per_kcal:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Added Sugars'] = score_list

In [25]:
### Saturated Fats评分
saturated_fats_per_kcal = (HEI_evaluate_df['DRTSFAT']) * 9 / HEI_evaluate_df['DRTKCAL']
score_list = []
max = 0.16
min = 0.08
points = 10
for i in saturated_fats_per_kcal:
    if i <= min:
        score = points    
        score_list.append(score)
    elif i >= max:
        score = 0
        score_list.append(score)
    else:
        score = points*(max - i)/(max - min)
        score_list.append(score)

score_in_HEI['Saturated Fats'] = score_list

In [26]:
# 求总分
score_sum_HEI = score_in_HEI
score_sum_HEI['diet_score'] = score_in_HEI.drop(labels='SEQN', axis=1).sum(axis=1)

In [27]:
score_in_HEI

,SEQN,Total Fruits,Whole Fruit,Total Vegetables,Greens and Beans,Whole Grains,Dairy,Total Protein,Seafood and Plant Proteins,Fatty Acids,Refined Grains,Sodium,Added Sugars,Saturated Fats,diet_score
0,51624.0,5.000000,5.000000,2.537397,0.000000,0.000000,10.000000,2.944786,0.000000,0.000000,4.088995,0.0,7.163225,0.000000,36.734403
1,51625.0,2.176916,4.273964,0.958920,1.298152,5.722809,9.715135,5.000000,0.324538,4.405965,3.368372,0.0,5.919490,7.915313,51.079572
2,51626.0,0.913879,0.000000,0.610482,0.000000,10.000000,10.000000,5.000000,0.000000,1.484063,7.076481,0.0,6.716444,2.195446,43.996795
3,51627.0,1.878136,0.745589,1.694834,0.000000,7.598273,10.000000,3.628946,0.000000,0.882023,2.714221,0.0,10.000000,0.000000,39.142022
4,51628.0,0.000000,0.000000,5.000000,5.000000,0.000000,3.450888,5.000000,0.000000,7.273308,0.000000,0.0,1.186641,6.730166,33.641004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9749,62156.0,1.319234,0.000000,5.000000,5.000000,3.293676,3.833899,2.595918,4.710114,0.000000,10.000000,0.0,0.716692,0.000000,36.469534
9750,62157.0,5.000000,5.000000,0.912259,0.000000,0.000000,3.026243,5.000000,0.732746,8.278866,0.000000,0.0,7.010706,8.593955,43.554774
9751,62158.0,1.112597,0.000000,4.652689,5.000000,0.000000,1.959482,5.000000,0.000000,2.162480,2.888304,0.0,0.854597,2.975407,26.605557
9752,62159.0,3.325571,4.431170,2.431591,0.000000,0.153466,0.077735,0.000000,0.000000,1.032641,8.651641,0.0,10.000000,3.540656,33.644471


In [28]:
score_in_HEI.to_excel('E:/work/Paper/NHANES/0910Analysis Data/Final data/DGA_HEI_score.xlsx')